# Información de Campos del dataframe

| Field   | Comment                                                                 |
|---------|-------------------------------------------------------------------------|
| START   | The start timestamp of the template execution in milliseconds           |
| END     | The end timestamp of the template execution in milliseconds             |
| TIMEOUT | Indicates if the execution exceeded a predefined time limit             |
| system  | The name of the system used (e.g., PIONIER)                             |
| procname| The process name associated with the template execution                 |
| TPL_ID  | The filename of the corresponding template file                         |
| ERROR   | Indicates if there was an error during execution                        |
| Aborted | Indicates if the template execution was aborted (manually or because an error) |
| SECONDS | The duration of the template execution in seconds                       |
| TEL     | The class of telescope used in the observation, in this dataset it is only AT |

# Información de Campos de Log (metadata de los Log)

| Field       | Description                                                                                           |
|-------------|-------------------------------------------------------------------------------------------------------|
| @timestamp  | The timestamp of the log entry in milliseconds.                                                       |
| system      | The name of the system (e.g., PIONIER) from which the log entry originates.                           |
| hostname    | The hostname of the machine where the log entry was generated.                                        |
| loghost     | The host of the logging system that generated the entry.                                              |
| logtype     | Type of the log entry (e.g., LOG, FEVT, ERR), indicating its nature such as general log, event, or error. |
| envname     | The environment name where the log was generated, providing context for the log entry.                |
| procname    | The name of the process that generated the log entry.                                                 |
| procid      | The process ID associated with the log entry.                                                         |
| module      | The module from which the log entry originated, indicating the specific part of the system.           |
| keywname    | Name of any keyword associated with the log entry, if applicable. It is always paired with keywvalue. |
| keywvalue   | Value of the keyword mentioned in keywname, if applicable.                                            |
| keywmask    | Mask or additional context for the keyword, if applicable.                                            |
| logtext     | The actual text of the log entry, providing detailed information about the event or action.           |
| trace_id    | A unique identifier associated with each log entry, corresponds to id in metadata table.              |

In [1]:
import os
import pandas as pd
import subprocess
from urllib.request import urlretrieve

REPO_URL = 'https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'

if 'COLAB_RELEASE_TAG' in os.environ.keys():
    PATH = 'data/'  # Convenient name to be Colab compatible
else:
    PATH = 'D:/Ufro/11 nivel/Lab modelacion 2/Trabajo_paramal/collaborations-UFRO-2024-2s/data'  # Local directory to your system

def load_dataset(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-traces.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_inst = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_subs = pd.read_parquet(f'{PATH}/{fname}')

    fname = f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_tele = pd.read_parquet(f'{PATH}/{fname}')

    all_traces = [df_inst, df_subs, df_tele]

    df_all = pd.concat(all_traces)
    df_all.sort_values('@timestamp', inplace=True)
    df_all.reset_index(drop=True, inplace=True)

    return df_all

def load_trace(INSTRUMENT, RANGE, trace_id):
    df_all = load_dataset(INSTRUMENT, RANGE)
    df_all = df_all[df_all['trace_id'] == trace_id]
    return df_all

def load_meta(INSTRUMENT, RANGE):
    fname = f'{INSTRUMENT}-{RANGE}-meta.parket'
    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df_meta = pd.read_parquet(f'{PATH}/{fname}')
    return df_meta

# Función para generar el código PlantUML para un solo usuario
def generate_plantuml_code_single_user(meta_row, trace_row):
    uml_code = "@startuml\n"
    uml_code += "actor User\n"
    uml_code += "participant System\n"
    uml_code += "participant Instrument\n"
    uml_code += "participant Subsystem\n\n"

    start_time = meta_row['START']
    end_time = meta_row['END']
    procname = meta_row['procname']
    status = 'success' if not meta_row['ERROR'] else 'failure'
    logtext = trace_row['logtext']

    uml_code += f"User -> System: Send Command ({procname})\n"
    uml_code += "activate System\n"
    uml_code += f"System -> Instrument: Forward Command ({procname})\n"
    uml_code += "activate Instrument\n"
    uml_code += f"Instrument -> Subsystem: Execute Command ({procname})\n"
    uml_code += "activate Subsystem\n\n"

    if status == 'success':
        uml_code += f"Subsystem -> Instrument: Return Success ({start_time}, {end_time})\n"
    else:
        uml_code += f"Subsystem -> Instrument: Return Failure ({start_time}, {end_time})\n"
    
    uml_code += "deactivate Subsystem\n\n"
    uml_code += f"Instrument -> System: Return Status ({status})\n"
    uml_code += "deactivate Instrument\n\n"
    uml_code += f"System -> User: Return Status ({status})\n"
    uml_code += "deactivate System\n\n"
    uml_code += f"note right of System: Log Entry - {logtext}\n"

    uml_code += "@enduml"
    return uml_code

# Cargar los datos de observación y metadatos
INSTRUMENT = "PIONIER"
RANGE = "1w"
TRACE_ID = 30  # Ejemplo de trace_id, ajusta según tus datos

df_meta = load_meta(INSTRUMENT, RANGE)
df_trace = load_trace(INSTRUMENT, RANGE, TRACE_ID)

# Verificar las columnas de los DataFrames
print("Columnas en df_meta:", df_meta.columns)
print("Columnas en df_trace:", df_trace.columns)

# Seleccionar una fila del DataFrame para el usuario

meta_row=df_meta.iloc[TRACE_ID]


if 'trace_id' in df_trace.columns:
    trace_row = df_trace.iloc[0]
else:
    print("La columna 'trace_id' no está presente en df_trace. Verifica el nombre de la columna.")

# Generar el código PlantUML para un solo usuario
uml_code = generate_plantuml_code_single_user(meta_row, trace_row)

# Guardar el código PlantUML en un archivo
with open('dynamic_diagram.puml', 'w') as file:
    file.write(uml_code)

# Renderizar el diagrama usando PlantUML a través de subprocess
subprocess.run(['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'])

Columnas en df_meta: Index(['START', 'END', 'TIMEOUT', 'system', 'procname', 'TPL_ID', 'ERROR',
       'Aborted', 'SECONDS', 'TEL'],
      dtype='object')
Columnas en df_trace: Index(['@timestamp', 'system', 'hostname', 'loghost', 'logtype', 'envname',
       'procname', 'procid', 'module', 'keywname', 'keywvalue', 'keywmask',
       'logtext', 'trace_id'],
      dtype='object')


CompletedProcess(args=['java', '-jar', 'plantuml.jar', 'dynamic_diagram.puml'], returncode=1)

In [2]:
from PIL import Image

In [3]:
df_meta.iloc[TRACE_ID]

START       2019-04-02 10:18:42.266000
END         2019-04-02 10:18:55.430000
TIMEOUT                          False
system                         PIONIER
procname                       bob_ins
TPL_ID           PIONIER_gen_tec_setup
ERROR                            False
Aborted                          False
SECONDS                           13.0
TEL                                 AT
Name: 30, dtype: object

In [4]:
trace_id=10

In [5]:
df_trace[df_trace['trace_id'] == trace_id].iloc[0]

IndexError: single positional indexer is out-of-bounds